記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し，辞書オブジェクトとして格納せよ
https://ja.wikipedia.org/wiki/Help:%E6%97%A9%E8%A6%8B%E8%A1%A8

In [1]:
from collections import OrderedDict
from pprint import pprint
import re

import pandas as pd

In [2]:
def extract_by_title(title):
    df_wiki = pd.read_json('jawiki-country.json', lines=True)
    return df_wiki[(df_wiki['title'] == title)]['text'].values[0]

In [3]:
wiki_body = extract_by_title('イギリス')

In [4]:
# rを先頭にするとraw string でエスケープシーケンス無視
# 3重クォートで途中改行無視
# re.VERBOSEオプションを使うことによって、空白とコメント無視
# re.MULTILINEで複数行に対して検索
# re.DOTALLで改行もワイルドカード対象
# 非貪欲マッチにすることで、短い文字列を検索
basic = re.search(r'''
                    ^\{\{基礎情報.*?\n  #検索語句(\はエスケープ処理)、非キャプチャ、非貪欲
                    (.*?)              #任意の文字列
                    \}\}               #検索語句(\はエスケープ処理)
                    $                  #文字列の末尾
                    ''', wiki_body, re.MULTILINE+re.VERBOSE+re.DOTALL)
pprint(basic.group(1))

('|略名 = イギリス\n'
 '|日本語国名 = グレートブリテン及び北アイルランド連合王国\n'
 '|公式国名 = {{lang|en|United Kingdom of Great Britain and Northern '
 'Ireland}}<ref>英語以外での正式国名:<br/>\n'
 '*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu '
 'Thuath}}（[[スコットランド・ゲール語]]）<br/>\n'
 '*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd '
 'Iwerddon}}（[[ウェールズ語]]）<br/>\n'
 '*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na '
 'hÉireann}}（[[アイルランド語]]）<br/>\n'
 '*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon '
 'Glédh}}（[[コーンウォール語]]）<br/>\n'
 '*{{lang|sco|Unitit Kinrick o Great Breetain an Northren '
 'Ireland}}（[[スコットランド語]]）<br/>\n'
 '**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin '
 'Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin '
 'Airlann}}（アルスター・スコットランド語）</ref>\n'
 '|国旗画像 = Flag of the United Kingdom.svg\n'
 '|国章画像 = [[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]\n'
 '|国章リンク = （[[イギリスの国章|国章]]）\n'
 '|標語 = {{lang|fr|Dieu et mon droit}}<br/

In [5]:
# rを先頭にするとraw string でエスケープシーケンス無視
# 3重クォートで途中改行無視
# re.VERBOSEオプションを使うことによって、空白とコメント無視
# re.MULTILINEで複数行に対して検索
# re.DOTALLで改行もワイルドカード対象
# 非貪欲マッチにすることで、短い文字列を検索
templates = OrderedDict(re.findall(r'''
                          ^\|         # \はエスケープ処理、非キャプチャ
                          (.+?)       # キャプチャ対象(key)、非貪欲
                          \s*         # 空白文字0文字以上
                          =           # 検索語句、非キャプチャ
                          \s*         # 空白文字0文字以上
                          (.+?)       # キャプチャ対象(Value)、非貪欲
                          (?:         # キャプチャ対象外のグループ開始
                            (?=\n\|)  # 改行(\n)+'|'の手前(肯定の先読み)
                          | (?=\n$)   # または、改行(\n)+終端の手前(肯定の先読み)
                          )           # キャプチャ対象外のグループ終了
                         ''', basic.group(1), re.MULTILINE+re.VERBOSE+re.DOTALL))
pprint(templates)

OrderedDict([('略名', 'イギリス'),
             ('日本語国名', 'グレートブリテン及び北アイルランド連合王国'),
             ('公式国名',
              '{{lang|en|United Kingdom of Great Britain and Northern '
              'Ireland}}<ref>英語以外での正式国名:<br/>\n'
              '*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn '
              'mu Thuath}}（[[スコットランド・ゲール語]]）<br/>\n'
              '*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd '
              'Iwerddon}}（[[ウェールズ語]]）<br/>\n'
              '*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart '
              'na hÉireann}}（[[アイルランド語]]）<br/>\n'
              '*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon '
              'Glédh}}（[[コーンウォール語]]）<br/>\n'
              '*{{lang|sco|Unitit Kinrick o Great Breetain an Northren '
              'Ireland}}（[[スコットランド語]]）<br/>\n'
              '**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin '
              'Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin '
           